In [1]:
import pandas as pd
import glob
import os

In [31]:
import glob
import os
import pandas as pd

path = "datosreto/Datos looks/*.csv"
files = glob.glob(path)

dfs = {}

for file in files:
    base = os.path.splitext(os.path.basename(file))[0]
    if base == 'product_2':
        dfs[base] = pd.read_csv(file, sep=';')
    else:
        dfs[base] = pd.read_csv(file, sep=',')


In [49]:
df_productos = pd.DataFrame()
for i in range(len(dfs['product_2'])):
    producto = pd.DataFrame(dfs['product_2'].iloc[i]).T
    df_merged = producto.merge(
        dfs['product_variant'][['color_id', 'season', 'size_id', 'product_id']],
        left_on='id',
        right_on='product_id',
        how='left'
    )
    df_merged = df_merged.drop(columns=['Unnamed: 0', 'product_id'])
    df_merged = df_merged.merge(
        dfs['color'][['id', 'name']],
        left_on='color_id',
        right_on='id',
        how='left'
    )
    df_merged = df_merged.drop(columns=['id_y', 'color_id'])
    df_merged = df_merged.merge(
        dfs['size'][['id', 'lookiero']],
        left_on='size_id',
        right_on='id',
        how='left'
    )
    df_merged = df_merged.drop(columns=['id', 'size_id'])
    df_merged = df_merged.merge(
        dfs['brand'][['id', 'name']],
        left_on='brand_id',
        right_on='id',
        how='left'
    )
    df_merged = df_merged.drop(columns=['id', 'brand_id'])
    df_merged = df_merged.merge(
        dfs['product_feature_value'][['id', 'feature_value_id', 'product_id']],
        left_on='id_x',
        right_on='product_id',
        how='left'
    )
    df_merged = df_merged.drop(columns=['product_id'])
    df_merged = df_merged.rename(columns={'id_x': 'id_producto'})
    df_merged = df_merged.merge(
        dfs['feature_value'][['id', 'feature_id', 'value']],
        left_on='feature_value_id',
        right_on='id',
        how='left'
    )
    df_merged = df_merged.drop(columns=['id_y'])
    df_merged = df_merged[['id_producto', 'title', 'season', 'name_x', 'lookiero', 'name_y','feature_id', 'value']]
    df_merged = df_merged.merge(
        dfs['feature'][['id', 'name']],
        left_on='feature_id',
        right_on='id',
        how='left'
    )
    df_merged = df_merged.drop(columns=['feature_id', 'id'])
    df_merged = df_merged.rename(columns={'name_x': 'color', 'name_y': 'marca', 'name':'caracteristica1', 'value':'caracteristica2'})
    cols = [
        "title", "season", "color", "lookiero",
        "marca", "caracteristica1", "caracteristica2"
    ]

    def unir_valores_unicos(s):
        return ", ".join(s.dropna().astype(str).unique())

    producto = (
        df_merged
        .groupby("id_producto")[cols]
        .agg(unir_valores_unicos)
        .reset_index()
    )
    producto = pd.DataFrame(producto)
    df_productos = pd.concat([df_productos, producto], ignore_index=True)


In [50]:
df_productos

,id_producto,title,season,color,lookiero,marca,caracteristica1,caracteristica2
0,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,"black, beige, rust","{""size"": ""XL"", ""format"": ""XL""}, {""size"": ""M"", ...",BRAND119,"top_type, finishing, adventurous, closing, wea...","cardigan, ribbed, two, without_closing, cold_s..."
1,6e6b4255-006a-4d13-b312-3b1426290d8e,Poly Pant deeluxe,7,red,"{""size"": ""M"", ""format"": ""M""}, {""size"": ""S"", ""f...",BRAND117,"down_part_type, adventurous, bottom, closing, ...","cigarette, three, two, straight_pants, fly, wa..."
2,6549c1b1-989e-432a-bdc9-b603c438b310,Onleden Sweater knt,9,"yellow, ecru, blue_dark","{""size"": ""M"", ""format"": ""M""}, {""size"": ""XS"", ""...",BRAND119,"top_type, adventurous, closing, weather, neckl...","sweaters, two, without_closing, warm_season, c..."
3,dace6963-7e84-4cef-a153-3c6fb29cc7c3,Cerise Jacket print,8,"blue_dark, black","{""size"": ""S"", ""format"": ""S""}, {""size"": ""XL"", ""...",BRAND15,"outside_type, adventurous, closing, weather, d...","bomber, three, zip, warm_season, sporty_band, ..."
4,d6679919-7a49-4edd-bd04-982289d0e8ac,Auttum Shirt hippie,8,"green_dark, camel","{""size"": ""L"", ""format"": ""L""}, {""size"": ""S"", ""f...",BRAND4,"top_type, alerts, adventurous, closing, weathe...","shirt, breastfeeding, two, three, hidden_butto..."
...,...,...,...,...,...,...,...,...
2927,c70a703e-dc16-4ac4-8c14-fbba5660bcb1,Gumm Pant place,9,"khaki, brown_light, black","{""size"": ""XS"", ""format"": ""XS""}, {""size"": ""L"", ...",BRAND84,"down_part_type, adventurous, bottom, closing, ...","cargo, three, straight_pants, fly, warm_season..."
2928,9f3acaac-17b3-43a4-ae09-1677293d8ace,Kentucky Shirt melange,9,blue,"{""size"": ""XXL"", ""format"": ""XXL""}, {""size"": ""XL...",BRAND151,"top_type, alerts, adventurous, closing, weathe...","shirt, breastfeeding, two, hidden_buttoning, w..."
2929,50701dda-a3f4-4b59-8698-da3757382574,Lines Top cerise,9,"blue_light, green_light, yellow, pink","{""size"": ""M"", ""format"": ""M""}, {""size"": ""L"", ""f...",BRAND15,"top_type, adventurous, closing, weather, detai...","tops, two, without_closing, warm, ruffles, v_n..."
2930,22efc184-5ecb-401d-8dea-9569ceae492d,Jara Short pant,9,"red, purple","{""size"": ""L"", ""format"": ""L""}, {""size"": ""S"", ""f...",BRAND15,"down_part_type, adventurous, closing, weather,...","shorts, three, without_closing, warm, pleats, ..."


In [ ]:
# Supongamos que df_productos['color'] ya tiene listas con los colores separados
colores_totales = []
for lista_colores in df_productos['color']:
    colores_totales.extend(lista_colores)

total_colores = len(colores_totales)
print(total_colores)  


39125
